# Supply Chain Route Assignment Optimization

## Setup Prerequisites

In [1]:
pip install pandas numpy pulp --user --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


## Import Libraries

In [2]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpConstraint

## Load and Clean Data

In [3]:
def load_and_clean_data(file_path):
    """
    Load and preprocess the supply chain data from an Excel file.
    
    Parameters:
        file_path (str): Path to the Excel file containing all data tables.
    
    Returns:
        dict: A dictionary containing cleaned and consolidated data DataFrames.
    """
    data = pd.read_excel(file_path, sheet_name=None)

    # Data cleaning and preprocessing for each table
    cleaned_data = {}
    for table_name, table_data in data.items():
        cleaned_data[table_name] = table_data.dropna()
    
    return cleaned_data

## Route Assignment Optimization

In [4]:
def route_assignment_optimization(data):
    """
    Perform route assignment optimization.

    Parameters:
        data (dict): A dictionary containing cleaned and consolidated data DataFrames.

    Returns:
        LpProblem: The PuLP optimization problem.
    """
    # Extract relevant data from the consolidated DataFrames
    order_data = data['OrderList']
    freight_rates = data['FreightRates']
    wh_costs = data['WhCosts']
    wh_capacities = data['WhCapacities']
    products_per_plant = data['ProductsPerPlant']
    vmi_customers = data['VmiCustomers']
    plant_ports = data['PlantPorts']

    # Define decision variables
    route_assignments = LpVariable.dicts("RouteAssignment",
                                        ((order, route) for order in order_data['Order ID'] for route in freight_rates['Carrier']),
                                        cat="Binary")

    # Define the optimization problem
    problem = LpProblem("RouteAssignmentOptimization", LpMinimize)

    # Define the objective function
    costs = []
    for order in order_data['Order ID']:
        for route in freight_rates['Carrier']:
            cost = order_data.loc[order_data['Order ID'] == order, 'Weight'].values[0] * freight_rates.loc[
                freight_rates['Carrier'] == route, 'minimum cost'].values[0]
            costs.append(route_assignments[(order, route)] * cost)

    problem += lpSum(costs), "Total Cost"

    # Add constraints (rest of the code remains the same)


## Perform Optimization

In [ ]:
if __name__ == '__main__':
    # Load and preprocess the supply chain data
    data = load_and_clean_data("C:/Users/neluw/Desktop/Data Science Projects/Supply chain logisitcs problem.xlsx")

    # Perform route assignment optimization
    problem = route_assignment_optimization(data)

    # Solve the optimization problem
    problem.solve()

    # Interpret the results
    for order, route in route_assignments:
        if route_assignments[(order, route)].varValue == 1:
            print(f"Assign Order {order} to Route {route}")

    print(f"Total Cost: ${problem.objective.value()}")